# Limpieza de Datos Dataset Movimientos-20190716

In [ ]:
import pandas as pd
from utils_functions import  upload_to_bucket

# Importar Dataset

#### Importar el dataset y observar los primeros registros con head y verificar los tipos de datos con el método info.

In [ ]:
movimientos_col_names = [
    'idb', 'id_producto', 'fecha_trans', 'stock_unidades', 'venta_unidades', 'precio_unitario',
    'cd_abast', 'sigue_prod_en_rol'
]

In [ ]:
path = '../data/VMI_Movimientos-20190716.txt'
movimientos_df = pd.read_csv(path,
                 delimiter = '\t',
                 header=None,
                 names=movimientos_col_names)

In [ ]:
movimientos_df.head()

In [ ]:
movimientos_df.info()

#### Ajustes al dataframe
- Convertir la columna fecha_trans a tipo de dato datetime.
- Crear columnas de año, mes y día apartir de la columna fecha.
- Convertir la columna sigue_prod_en_rol a tipo de dato string.
- Verificar por valores nulos y repetidos.
- Convertir la columna sigue_prod_en_rol a variable numerica donde 1 corresponde a SI y 0 corresponde a NO.
- Obtener las fechas máximas y minimas de transaciones
- Obtener una análisis descriptivo de los datos.

In [ ]:
movimientos_df['fecha_trans'] = pd.to_datetime(movimientos_df['fecha_trans'], format='%Y-%m-%d')
movimientos_df['year'] = movimientos_df['fecha_trans'].dt.year
movimientos_df['month'] = movimientos_df['fecha_trans'].dt.month
movimientos_df['weekday'] = movimientos_df['fecha_trans'].dt.weekday

In [ ]:
movimientos_df.isnull().sum(axis=0)

In [ ]:
movimientos_df.duplicated().sum()

In [ ]:
print(f"La cantidad de productos únicos en el dataset son: {movimientos_df.id_producto.nunique()}")

In [ ]:
# Verificar si existen valores duplicados para el subset: idb, id_producto, fecha_trans, precio_unitario
duplicated_mask = movimientos_df.duplicated(subset=['idb', 'id_producto', 'precio_unitario', 'fecha_trans'], keep='first')
num_duplicated_records = sum(duplicated_mask)
print(f"El número de registros duplicados es: {num_duplicated_records}")

In [ ]:
movimientos_df['sigue_prod_en_rol'] = movimientos_df['sigue_prod_en_rol'].str.lower().apply(lambda x: 1 if x == 'si' else 0)

In [ ]:
max_date = movimientos_df['fecha_trans'].max()
min_date = movimientos_df['fecha_trans'].min()

print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

In [ ]:
movimientos_df.describe()

In [ ]:
# Verificar cuantos valores negativos tenemos en la columna venta_unidades
ventas_unidades_negativas = movimientos_df[movimientos_df['venta_unidades'] < 0]
print(f"Las ventas en unidades con valores negativos son en total: {ventas_unidades_negativas.id_producto.count()}")

In [ ]:
# Verificar cuantos valores negativos tenemos en la columna stock unidades
stock_unidades_negativas = movimientos_df[movimientos_df['stock_unidades'] < 0]
print(f"El total de inventario en unidades negativos son en total: {stock_unidades_negativas.id_producto.count()}")

#### Observaciones dataset Movimientos-20190716

1. El conjunto de datos contiene información sobre los movimientos de productos desde el 15 de julio de 2016 hasta el 14 de julio de 2019.
2. El número de registros por año es el siguiente:
        2016: 1,177,264
        2017: 2,343,400
        2018: 2,914,155
        2019: 1,912,998
3. El conjunto de datos consta de 8 columnas identificadas de la siguiente manera:
        - idb: identificador interno de la ubicación
        - id_producto: identificador interno del producto
        - fecha: fecha de la transacción (un día antes de la fecha actual)
        - stock_unidades: número de unidades de producto en la ubicación
        - venta_unidades: número de unidades vendidas
        - precio_unitario: precio de venta unitario
        - cd_abast: identificador de la ubicación de la tienda que suministra el producto a la ubicación actual.
        - sigue_prod_en_rol: indica si el producto sigue en rol (SI) o no (NO) en la ubicación actual.
4. No se observan valores faltantes o duplicados. Esta es una tabla transaccional, por lo que puede haber múltiples registros para el mismo producto y ubicación con diferentes fechas de transacciones.
5. El valor medio de unidades de inventario es de 20.65585 unidades.
6. El valor medio de unidades vendidas es de 0.3133696 unidades.
7. La media del precio unitario es de 64.26050 pesos.
8. Se observan 580 registros que contienen valores negativos en la columna venta_unidades.
9. Se observan 45,483 registros que contienen valores negativos en la columna stock_unidades.
10. La cantidad de productos únicos en el conjunto de datos es de 92.



### Guardar el dataset limpio

In [ ]:
movimientos_df.to_csv('../clean_data/movimientos_clean.csv', index=False)

#### Subir csv a google cloud storage

In [ ]:
upload_to_bucket('deodorants', '../clean_data/movimientos_clean.csv', 'movimientos')